# Tutaj raczej w ramach ciekawostki predykcje długości skoku wybranego zawodnika

Skok jest przewidywany za pomocą regresji liniowej na podstawie prędkości na progu, rekompensaty za wiatr,
punktu konstrukcyjnego skoczni oraz rozmiaru skoczni.
Problemem może być jednak to, że do momentu wyjścia z progu nie znamy prędkości zawodnika. Możemy ją jedynie przewidzieć na podstawie długości najazdu albo oszacować na podstawie uzyskiwanych prędkości na danej skoczni w zależności od długości najazdu, ale nie mam takich informacji.

Należy pamiętać, że główny wpływ na długośc skoku ma forma zawodnika.
Model przewiduje odległość, jaką zawodnik powinien osiągnąć będąc w swojej optymalnej dyspozycji, czyli w której
oddał najwięcej skoków w badanym okresie. Dlatego średnie odchylenie standardowe może być wysokie. Model najlepiej będzie działał dla regularnych skoczków.

Dlatego świetne wyniki uzyskujemy dla Dawida Kubackiego, który jest jednym z najbardziej przewidywalnych skoczków.
Na przeciwnym biegunie jest Thomas Diethart, który był "skoczkiem jednego sezonu". Również dla Andreasa Wellingera w okresie 2020-2022 model nie przewidzi dobrze odległości, ponieważ rok 2021 to bardzo duży spadek formy tego zawodnika i skakanie na poziomie FIS Cup.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

In [2]:
comps = pd.read_csv("data/all_comps.csv")
names = pd.read_csv("data/all_names.csv")
results = pd.read_csv("data/all_results.csv")

In [3]:
merged = results.merge(comps, on="id")

In [4]:
def predict_jumper(name, down=2009, up=2022):
    """
    funkcja przewidująca długość skoku danego zawodnika
    name: nazwa skoczka w konwencji: nazwisko imię
    down: rok, od którego zaczynamy zbierać dane. Domyślnie 2009
    up: rok, do którego kończymy zbieranie danych. Domyślnie 2022
    """
    codex = names[names['name'] == name].iloc[0,1]
    dataset = merged[merged['codex_x'] == codex]
    dataset = dataset[['speed', 'wind_comp', 'k-point', 'hill_size_x', 'dist', 'season']].dropna()
    data = dataset[(dataset['season'] >= down) & (dataset['season'] <= up)]
    data = data[(data['speed'] != 0) & (data['dist'] != 0)]
    print(data[['speed', 'wind_comp', 'k-point', 'hill_size_x', 'dist']].corr())
    
    X = data.iloc[:, :4].values
    y = data.iloc[:, 4].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    y_pred = regressor.predict(X_test)
    
    print(y_pred)
    print(y_test)
    
    accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
    print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [6]:
predict_jumper('kubacki dawid')

                speed  wind_comp   k-point  hill_size_x      dist
speed        1.000000   0.206450  0.907265     0.900619  0.873870
wind_comp    0.206450   1.000000 -0.014914    -0.018326 -0.057024
k-point      0.907265  -0.014914  1.000000     0.994861  0.946657
hill_size_x  0.900619  -0.018326  0.994861     1.000000  0.951944
dist         0.873870  -0.057024  0.946657     0.951944  1.000000
[ 89.88408908 120.826798   100.87391458 123.19405828 128.3861032
 213.4894592  127.54912365  99.53380104 126.2235595  128.11949851
 100.73707073 126.07768028 123.02784161 121.03210012 126.45053545
 129.81341992 128.63906497 116.95710404 117.30994747 120.39735403
 131.70087307 126.54461007 124.49647957 128.62955876 129.33446073
 125.74600317 124.61706361 129.41827095 129.37271531 127.88397703
 122.01703051 120.8113166  127.54444481 129.1251906  127.56399223
 124.7892765  118.24119331 127.87829234 123.30347228 126.10006692
 117.95569095 116.720341   128.92873866 125.10219618 115.29252817
 118.523011

In [7]:
predict_jumper('diethart thomas')

                speed  wind_comp   k-point  hill_size_x      dist
speed        1.000000   0.411943  0.859856     0.865870  0.828813
wind_comp    0.411943   1.000000  0.197195     0.203765  0.143406
k-point      0.859856   0.197195  1.000000     0.994923  0.881732
hill_size_x  0.865870   0.203765  0.994923     1.000000  0.884882
dist         0.828813   0.143406  0.881732     0.884882  1.000000
[121.71304258 120.62891374  88.28447272 167.85135616 119.2284741
 118.72025747 124.11878999  89.49077828 113.98558409 114.01204432
 101.3839216   87.47151439 115.91523323 120.81447115 120.16463804
 112.05047368 124.20154602 118.87921292 119.27889442 125.46576094
 118.06417476 112.04866982 117.84060089  88.97642378 119.69381027
 121.24006773 125.92695214 127.07124454 117.97933889 121.70433608
  88.99720055 126.31362643 115.52959518  88.65738971 125.60253763
 166.98234816 118.15954803 120.7489717  123.37446919 110.37899394
 168.63208995 114.53842511 110.10041398 115.9313249   86.3143141
 115.4595093